<a href="https://colab.research.google.com/github/PranavDixit2/LungCancer/blob/main/CXR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow streamlit numpy pandas pillow opencv-python matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.5 MB/s eta 0:00:00


In [7]:
import os

# List files in the specified directory
directory_path = "/content/drive/MyDrive/Projects/300B Practicum"
print(os.listdir(directory_path))

['fine_tuned_model_resnet_updated.h5', 'Questions.gdoc', 'Practicum_Proposal_V1.docx']


In [9]:
from tensorflow.keras.models import load_model

# Load the model
model_path = "/content/drive/MyDrive/Projects/300B Practicum/fine_tuned_model_resnet_updated.h5"  # Update this path if using Google Drive
model = load_model(model_path)

# Validate the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,850,115 (90.98 MB)

 Trainable params: 15,240,449 (58.14 MB)

 Non-trainable params: 8,609,664 (32.84 MB)

 Optimizer params: 2 (12.00 B)

In [10]:
import numpy as np
from PIL import Image

def preprocess_image(image):
    # Convert to grayscale if necessary
    if image.mode != 'L':
        image = image.convert('L')

    # Resize image to 224x224 pixels
    image = image.resize((224, 224))

    # Convert to NumPy array and normalize
    image_array = np.array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    return image_array

In [11]:
def classify_image(image_array):
    prediction = model.predict(image_array)
    probability = prediction[0][0]  # Assuming binary classification
    return probability

In [18]:
import os
import numpy as np
import pandas as pd
import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt

# Load the fine-tuned ResNet model
model_path = "/content/drive/MyDrive/Projects/300B Practicum/fine_tuned_model_resnet_updated.h5"
model = load_model(model_path)

# Function to preprocess the uploaded image
def preprocess_image(img):
    img = img.resize((224, 224))  # Resize to the input size of the model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess for ResNet
    return img_array

# Function to make predictions
def predict(image_array):
    predictions = model.predict(image_array)
    return predictions

# Streamlit app layout
st.title("Lung Cancer Classification App")
st.write("Upload a lung scan image to classify it as cancerous or non-cancerous.")

# File uploader for images
uploaded_file = st.file_uploader("Choose a lung scan image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    img = image.load_img(uploaded_file, target_size=(224, 224))
    st.image(img, caption='Uploaded Image', use_column_width=True)

    # Preprocess the image
    img_array = preprocess_image(img)

    # Make predictions
    predictions = predict(img_array)
    class_labels = ['Non-Cancerous', 'Cancerous']  # Adjust based on your model's output
    predicted_class = class_labels[np.argmax(predictions)]

    # Display the prediction
    st.write(f"Prediction: {predicted_class}")
    st.write(f"Confidence: {np.max(predictions) * 100:.2f}%")

    # Optional: Display prediction probabilities
    st.bar_chart(predictions[0])

2025-04-01 23:09:08.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:09:08.448 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:09:08.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:09:08.450 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:09:08.451 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:09:08.452 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:09:08.453 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:09:08.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar